### Question 4

In [1]:
# 1)
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("ReadFromData1") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("Word")
)

In [2]:
# 2)
hashtags = words.where(words.Word.like('#%'))
hashtags.printSchema()

root
 |-- Word: string (nullable = true)



In [3]:
# 3)
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
query = hashtags \
    .writeStream \
    .queryName("Query") \
    .outputMode("append") \
    .format("memory") \
    .start()

In [4]:
query.stop()

In [5]:
part_3 = sqlContext.sql("SELECT * FROM Query")
part_3.show()

+--------------------+
|                Word|
+--------------------+
| #trippolisontheroad|
|     #danipolisviaja|
|            #morning|
|           #Saturday|
|              #diner|
|                 #VT|
|          #breakfast|
|#nucorpsofcadetsring|
|               #ring|
|            #college|
|            #hiring!|
|                #job|
|             #Retail|
|                #Job|
|               #Jobs|
|             #Hiring|
|             #Retail|
|                #Job|
|               #Jobs|
|             #Hiring|
+--------------------+
only showing top 20 rows



In [6]:
# 4)
part_4 = sqlContext.sql("SELECT Word, COUNT(Word) AS Count FROM Query GROUP BY Word ORDER BY Count DESC LIMIT 10")
part_4.show()

+------------+-----+
|        Word|Count|
+------------+-----+
|     #Hiring|   22|
|       #Jobs|   18|
|        #Job|   16|
|  #CareerArc|   14|
|        #job|    8|
|       #job?|    6|
|     #Retail|    4|
|#Hospitality|    4|
|   #Veterans|    3|
|    #hiring!|    3|
+------------+-----+

